# iprPy.Database tar Archive Access Functions

- - -

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2017-05-03

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The Database class offers a number of methods for finding, accessing, changing and deleting the tar calculation archives associated with records. These functions all follow similar conventions on how they handle the information.

### add_tar(self, record=None, name=None, style=None, root_dir=None)

Archives a folder as a zipped tar file and saves it to the database associated with a particular record. The folder being archived is identified by having a name matching the record's name, and in being located in the specified root_dir. Issues an error if a matching record is not found, or if multiple matching records are found.

Aruments:

- __record__ is an iprPy.Record object. Cannot be given with the name and style.

- __name__ is the name of the record associated with the tar file.

- __style__ is the record style for the record associated with the tar file. Can be given with name if the same record name may exist in multiple styles.

- __root_dir__ specifies the root directory for finding the folder to archive. If not given, the current working directory is used.

### delete_tar(self, record=None, name=None, style=None)

Deletes a tar archive associated with a record. Issues an error if a matching record is not found, or if multiple matching records are found. __Deleting a tar is permanent__.

Aruments:

- __record__ is an iprPy.Record object. Cannot be given with the other arguments.

- __name__ is the name of a record in the database.

- __style__ is a record style. Can be given with name if the same record name may exist in multiple styles.


### get_tar(self, record=None, name=None, style=None)

Retrieves the archived tar folder associated with a record. Issues an error if a matching record is not found, or if multiple matching records are found. The retrieved archive is returned as an open Python TarFile object.

Aruments:

- __record__ is an iprPy.Record object. Cannot be given with the other arguments.

- __name__ is the name of a record in the database.

- __style__ is a record style. Can be given with name if the same record name may exist in multiple styles.

### update_tar(self, record=None, name=None, style=None, root_dir=None)

Allows for the content stored in a record's tar archive to be replaced and updated. The folder being archived is identified by having a name matching the record's name, and in being located in the specified root_dir. Issues an error if a matching record is not found, or if multiple matching records are found.

Aruments:

- __record__ is an iprPy.Record object. Cannot be given with the name and style.

- __name__ is the name of the record associated with the tar file.

- __style__ is the record style for the record associated with the tar file. Can be given with name if the same record name may exist in multiple styles.

- __root_dir__ specifies the root directory for finding the folder to archive. If not given, the current working directory is used.

The underlying code can be found in [iprPy/database_functions.py](../../iprPy/database_functions.py).

## Demonstration

Library Imports

In [1]:
#Standard libraries
from __future__ import print_function
import os
import shutil

#https://github.com/usnistgov/DataModelDict
from DataModelDict import DataModelDict as DM

#https://github.com/usnistgov/iprPy
import iprPy

Define content for a LAMMPS-potential record by using DataModelDict to convert from JSON to XML.

In [2]:
model = DM('''{
    "LAMMPS-potential": {
        "potential": {
            "key": "dc4149ce-3592-4131-8683-ecf654d5a519", 
            "id": "1987--Ackland-G-J--Ag"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": {
            "element": "Ag", 
            "mass": 107.8682
        }, 
        "pair_style": {
            "type": "eam/fs"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Ag.eam.fs"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}''')

content = model.xml()

Generate a corresponding folder to be archived.

In [3]:
os.mkdir('1987--Ackland-G-J--Ag')
with open(os.path.join('1987--Ackland-G-J--Ag', 'test.txt'), 'w') as f:
    f.write('test file content!')

Initialize a local style database stored in the test directory.

In [4]:
dbase = iprPy.Database('local', 'test')

Add the record to the database by providing name, style and content. A Record object is initialized based on the arguments and returned.

In [5]:
record = dbase.add_record(style='LAMMPS-potential', name='1987--Ackland-G-J--Ag', content=content)

Add the associated folder to the database associated with the record.

In [6]:
dbase.add_tar(record=record)

Retrieve the record as a TarFile object.

In [7]:
tar = dbase.get_tar(record=record)

print(tar.getnames())
tar.close()

['1987--Ackland-G-J--Ag', '1987--Ackland-G-J--Ag/test.txt']


Modify the local folder by adding test2.txt

In [8]:
with open(os.path.join('1987--Ackland-G-J--Ag', 'test2.txt'), 'w') as f:
    f.write('test file #2 content!')

Update the tar archive, then get it from the database to show that it has been changed

In [9]:
dbase.update_tar(record=record)

tar = dbase.get_tar(record=record)

print(tar.getnames())
tar.close()

['1987--Ackland-G-J--Ag', '1987--Ackland-G-J--Ag/test.txt', '1987--Ackland-G-J--Ag/test2.txt']


Delete the archive

In [10]:
dbase.delete_tar(record=record)

try:
    tar = dbase.get_tar(record=record)
except:
    print('No tar file')

No tar file


Cleanup

In [11]:
dbase.delete_record(name='1987--Ackland-G-J--Ag')
shutil.rmtree('1987--Ackland-G-J--Ag')

- - -

__Docs Navigation:__

Tutorial:

1. [Basics](../tutorial/1 Basics.ipynb)

Reference:

- [iprPy](../reference/iprPy.ipynb)

- [iprPy.calculations](../reference/iprPy.convert.ipynb)

- [iprPy.databases](../reference/iprPy.databases.ipynb)

- [iprPy.highthroughput](../reference/iprPy.highthroughput.ipynb)

- [iprPy.input](../reference/iprPy.input.ipynb)

- [iprPy.prepare](../reference/iprPy.prepare.ipynb)

- [iprPy.records](../reference/iprPy.records.ipynb)

- [iprPy.tools](../reference/iprPy.tools.ipynb)